In [1]:
# Check if curve_db is loaded, if not load it
try:
    curve_db
    print(f"✓ curve_db is already loaded with {len(curve_db)} isogeny classes")
except NameError:
    print("Loading curve database...")
    curve_db = load('Curve Database (Conductor < 100 000)/curve_database_c1_to_100000.sobj')
    print(f"✓ Loaded curve_db with {len(curve_db)} isogeny classes")
print()

Loading curve database...
✓ Loaded curve_db with 437226 isogeny classes



In [2]:
def elliptic_curve_from_phi(phi):
    """
    Compute the Weierstrass coefficients (a, b) for the elliptic curve:
    y² = x³ + ax + b
    
    where the coefficients are given in terms of φ by equation (70):
    a = -(9φ⁴ - 12φ³ + 30φ² - 12φ + 1) / 3
    b = -2(27φ⁶ - 54φ⁵ - 135φ⁴ + 180φ³ - 99φ² + 18φ - 1) / 27
    
    Parameters:
    -----------
    phi : numeric (int, float, or symbolic)
        The parameter φ
    
    Returns:
    --------
    tuple : (a, b)
        The two Weierstrass coefficients
    """
    # Compute coefficient a
    a_numerator = 9*phi**4 - 12*phi**3 + 30*phi**2 - 12*phi + 1
    a = -a_numerator / 3
    
    # Compute coefficient b
    b_numerator = 27*phi**6 - 54*phi**5 - 135*phi**4 + 180*phi**3 - 99*phi**2 + 18*phi - 1
    b = -2 * b_numerator / 27
    
    return a, b

def elliptic_curve_from_phi_sage(phi):
    """
    SageMath version that returns an actual EllipticCurve object.
    
    Parameters:
    -----------
    phi : numeric or symbolic
        The parameter φ
    
    Returns:
    --------
    EllipticCurve : Sage EllipticCurve object
    """
    from sage.all import EllipticCurve, QQ
    
    a, b = elliptic_curve_from_phi(phi)
    
    # Create elliptic curve in Weierstrass form [0, 0, 0, a, b]
    return EllipticCurve([0, 0, 0, a, b])

In [ ]:
phi = 15/2
elliptic_curve_from_phi_sage(phi).label()

In [7]:
data = load('phi_scan_results/(-500,500)_phi_scan_data')

In [8]:
phis = data['all_phis']

In [10]:
phi_to_iso = data['phi_to_iso']

In [14]:
isos = data['isogeny_classes']

In [23]:
from tqdm import tqdm

phis_class = []

for iso_class in tqdm(isos, desc='Parsing isogeny classes:'):
    phis_tmp = []
    for phi in phis:
        iso = phi_to_iso[phi]
        if iso == iso_class:
            phis_tmp.append(phi)
    phis_class.append(phis_tmp)

Parsing isogeny classes:: 100%|█████████████████████████████████████████████████████| 1296/1296 [01:22<00:00, 15.78it/s]


In [29]:
mults = [len(x) for x in phis_class]

In [37]:
phis_class[6]

[8, 7/71]

In [ ]:
phi_class

In [38]:
def phi_to_isogeny_class(phi):
    """
    Given a parameter φ, compute the corresponding elliptic curve
    and return its isogeny class label.
    
    Parameters:
    -----------
    phi : numeric
        The parameter φ (should result in a curve over Q)
    
    Returns:
    --------
    str : The isogeny class label (e.g., '37a')
    
    Raises:
    -------
    ValueError : If the curve is not defined over Q or has no Cremona label
    """
    from sage.all import EllipticCurve, QQ
    
    # Compute Weierstrass coefficients from phi (equation 70)
    a_numerator = 9*phi**4 - 12*phi**3 + 30*phi**2 - 12*phi + 1
    a = -a_numerator / 3
    
    b_numerator = 27*phi**6 - 54*phi**5 - 135*phi**4 + 180*phi**3 - 99*phi**2 + 18*phi - 1
    b = -2 * b_numerator / 27
    
    # Create elliptic curve
    E = EllipticCurve([0, 0, 0, a, b])
    
    # Ensure we're working over Q
    try:
        E_Q = E.change_ring(QQ)
    except (TypeError, ValueError):
        raise ValueError(f"Curve from φ={phi} is not defined over Q")
    
    # Convert to minimal model
    E_min = E_Q.minimal_model()
    
    # Get the Cremona label
    try:
        label = E_min.label()
    except (RuntimeError, LookupError) as e:
        raise ValueError(f"Could not find Cremona label for curve from φ={phi}: {e}")
    
    # Extract isogeny class from label (e.g., '37a2' -> '37a')
    i = len(label) - 1
    while i >= 0 and label[i].isdigit():
        i -= 1
    isogeny_class = label[:i+1]
    
    return isogeny_class

In [59]:
elliptic_curve_from_phi_sage(phis_class[0][1]).j_invariant()

8805624625/2312